In [55]:
from sklearn.model_selection import KFold,StratifiedKFold
import numpy as np
import pandas as pd
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict,cross_val_score,train_test_split
from sklearn.linear_model import ElasticNet
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV


In [56]:
pizza=pd.read_csv(r'..\Datasets\pizza.csv')
print(os.getcwd())
pizza.head()


c:\Users\dai.STUDENTSDC\AppData\DAI-1319\Machine learning\05-05-2025


,Promote,Sales
0,23,554
1,56,1339
2,34,815
3,25,609
4,67,1600


In [57]:
x, y = pizza.drop('Sales', axis=1), pizza[['Sales']]
el=ElasticNet()
kfold=KFold(n_splits=5,shuffle=True,random_state=25)
predication=cross_val_predict(estimator=el,X=x,y=y,cv=kfold)

predication


array([ 550.04809203, 1324.88527064,  801.80516292,  597.00792104,
       1573.30220061, 1934.05179147, 1090.08612561,  332.78656478,
        156.22978635, 1105.92487204, 1244.59215541, 2076.01640623,
       1666.47798548, 1816.28075469, 1580.52053091, 1037.54143761,
       1585.7181202 , 2156.39774604,  514.40564387])

In [58]:
results=cross_val_score(X=y,y=predication,cv=kfold, scoring='neg_root_mean_squared_log_error',estimator=el)
print(results)
results.mean()

[-0.07233106 -0.04596727 -0.0611785  -0.06260725 -0.03667683]


np.float64(-0.055752181720599835)

## using knn for solar dataset

In [59]:
sonar_df = pd.read_csv(r'..\Cases\Sonar\Sonar.csv')
sonar_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V52,V53,V54,V55,V56,V57,V58,V59,V60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [60]:
X= sonar_df.drop(columns=['Class'])
y= sonar_df['Class']
# le= LabelEncoder()
# y= le.fit_transform(y)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=25,stratify=y)

In [61]:
n_neighbors=[2,3,4,5,6]
scores=[]
kfold=KFold(n_splits=5,shuffle=True,random_state=25)
for neighbor in n_neighbors:
    knn=KNeighborsClassifier(n_neighbors=neighbor)
    results=cross_val_score(knn,X,y,cv=kfold)
    scores.append([neighbor,results,results.mean()])
scores_df=pd.DataFrame(scores,columns=['n_neighbors','Result array','mean_result'])
scores_df.sort_values(by='mean_result',ascending=False,inplace=True)
scores_df['Rank']=np.arange(1,len(n_neighbors)+1)
scores_df.head()  

,n_neighbors,Result array,mean_result,Rank
1,3,"[0.8333333333333334, 0.8095238095238095, 0.809...",0.807549,1
3,5,"[0.8571428571428571, 0.8095238095238095, 0.761...",0.802787,2
2,4,"[0.8333333333333334, 0.7619047619047619, 0.761...",0.783624,3
0,2,"[0.7857142857142857, 0.7380952380952381, 0.761...",0.759582,4
4,6,"[0.8095238095238095, 0.7142857142857143, 0.714...",0.750058,5


### GridSerachCV

In [62]:
from sklearn.model_selection import GridSearchCV

In [63]:
params={'n_neighbors':[2,3,4,5,6]}
knn=KNeighborsClassifier()
gcv=GridSearchCV(knn,param_grid=params,cv=kfold)
gcv.fit(X,y)
gcv.best_params_,gcv.best_score_

({'n_neighbors': 3}, np.float64(0.8075493612078978))

In [64]:
score_df=pd.DataFrame(gcv.cv_results_)
score_df.sort_values(by='rank_test_score',inplace=True)
score_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,0.002759,0.000370,0.011324,0.000578,3,{'n_neighbors': 3},0.833333,0.809524,0.809524,0.804878,0.780488,0.807549,0.016805,1
3,0.003036,0.001452,0.015951,0.002607,5,{'n_neighbors': 5},0.857143,0.809524,0.761905,0.780488,0.804878,0.802787,0.032165,2
2,0.003097,0.001264,0.014966,0.001247,4,{'n_neighbors': 4},0.833333,0.761905,0.761905,0.780488,0.780488,0.783624,0.026207,3
0,0.003285,0.000792,0.013336,0.002572,2,{'n_neighbors': 2},0.785714,0.738095,0.761905,0.707317,0.804878,0.759582,0.034448,4
4,0.003038,0.000915,0.015412,0.002507,6,{'n_neighbors': 6},0.809524,0.714286,0.714286,0.707317,0.804878,0.750058,0.046749,5


## SVM Radial

#### using K fold

In [65]:
sonar_df = pd.read_csv(r'..\Cases\Sonar\Sonar.csv')
sonar_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V52,V53,V54,V55,V56,V57,V58,V59,V60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [66]:
X= sonar_df.drop(columns=['Class'])
y= sonar_df['Class']
# le= LabelEncoder()
# y= le.fit_transform(y)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=25,stratify=y)

In [67]:
from sklearn.svm import SVC
kfold=KFold(n_splits=5,shuffle=True,random_state=25)
params={'C':np.linspace(0.001,5,10),
        'gamma':np.linspace(0.001,3,10)}
svm=SVC(kernel='rbf',probability=True,random_state=25)
gcv=GridSearchCV(svm,param_grid=params,cv=kfold)
gcv.fit(X,y)
gcv.best_params_,gcv.best_score_

({'C': np.float64(4.444555555555556),
  'gamma': np.float64(0.33422222222222225)},
 np.float64(0.8754936120789779))

In [68]:
score_df=pd.DataFrame(gcv.cv_results_)
score_df.sort_values(by='rank_test_score',inplace=True)
score_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
81,0.016761,0.000443,0.003299,0.000244,4.444556,0.334222,"{'C': 4.444555555555556, 'gamma': 0.3342222222...",0.928571,0.809524,0.785714,0.951220,0.902439,0.875494,0.065864,1
91,0.017542,0.001931,0.003369,0.000299,5.000000,0.334222,"{'C': 5.0, 'gamma': 0.33422222222222225}",0.928571,0.809524,0.785714,0.951220,0.902439,0.875494,0.065864,1
51,0.016954,0.000977,0.003221,0.000145,2.778222,0.334222,"{'C': 2.778222222222222, 'gamma': 0.3342222222...",0.928571,0.785714,0.809524,0.902439,0.902439,0.865738,0.056931,3
61,0.017870,0.001133,0.003253,0.000393,3.333667,0.334222,"{'C': 3.3336666666666663, 'gamma': 0.334222222...",0.928571,0.809524,0.785714,0.902439,0.902439,0.865738,0.056931,3
71,0.016338,0.000398,0.003220,0.000265,3.889111,0.334222,"{'C': 3.8891111111111107, 'gamma': 0.334222222...",0.928571,0.809524,0.785714,0.902439,0.902439,0.865738,0.056931,3


#### With Stratified Kfold

In [69]:
from sklearn.svm import SVC
skfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=25)
params={'C':np.linspace(0.001,5,10),
        'gamma':np.linspace(0.001,3,10)}
svm=SVC(kernel='rbf',probability=True,random_state=25)
gcv=GridSearchCV(svm,param_grid=params,cv=skfold)
gcv.fit(X,y)
gcv.best_params_,gcv.best_score_

({'C': np.float64(2.778222222222222), 'gamma': np.float64(1.6671111111111112)},
 np.float64(0.8799070847851336))

In [70]:
score_df=pd.DataFrame(gcv.cv_results_)
score_df.sort_values(by='rank_test_score',inplace=True)
score_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
55,0.025517,0.003350,0.003721,0.000374,2.778222,1.667111,"{'C': 2.778222222222222, 'gamma': 1.6671111111...",0.857143,0.928571,0.833333,0.926829,0.853659,0.879907,0.039865,1
95,0.025509,0.002316,0.003619,0.000418,5.000000,1.667111,"{'C': 5.0, 'gamma': 1.6671111111111112}",0.857143,0.928571,0.833333,0.926829,0.853659,0.879907,0.039865,1
75,0.024391,0.002809,0.003592,0.000325,3.889111,1.667111,"{'C': 3.8891111111111107, 'gamma': 1.667111111...",0.857143,0.928571,0.833333,0.926829,0.853659,0.879907,0.039865,1
65,0.023789,0.001983,0.003612,0.000413,3.333667,1.667111,"{'C': 3.3336666666666663, 'gamma': 1.667111111...",0.857143,0.928571,0.833333,0.926829,0.853659,0.879907,0.039865,1
85,0.025749,0.002808,0.003629,0.000392,4.444556,1.667111,"{'C': 4.444555555555556, 'gamma': 1.6671111111...",0.857143,0.928571,0.833333,0.926829,0.853659,0.879907,0.039865,1


In [71]:
boston=pd.read_csv(r'..\Cases\Boston Housing\Boston.csv')
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


## Grid Search using Pipeline

### 1. for boston dataset

In [72]:
x,y=boston.drop('medv',axis=1),boston['medv']

In [73]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

poly=PolynomialFeatures()
lr=LinearRegression()
pipe=Pipeline([("Poly",poly),("LR",lr)])
pipe.get_params()

{'memory': None,
 'steps': [('Poly', PolynomialFeatures()), ('LR', LinearRegression())],
 'transform_input': None,
 'verbose': False,
 'Poly': PolynomialFeatures(),
 'LR': LinearRegression(),
 'Poly__degree': 2,
 'Poly__include_bias': True,
 'Poly__interaction_only': False,
 'Poly__order': 'C',
 'LR__copy_X': True,
 'LR__fit_intercept': True,
 'LR__n_jobs': None,
 'LR__positive': False}

In [74]:
kfold=KFold(n_splits=5,shuffle=True,random_state=True)

In [75]:
params={"Poly__degree":[2,3,4]}
gcv=GridSearchCV(pipe,param_grid=params,cv=kfold)
gcv.fit(x,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=True, shuffle=True),
             estimator=Pipeline(steps=[('Poly', PolynomialFeatures()),
                                       ('LR', LinearRegression())]),
             param_grid={'Poly__degree': [2, 3, 4]})

In [76]:
gcv.best_params_,gcv.best_score_

({'Poly__degree': 2}, np.float64(0.8354513478407453))

In [77]:
el=ElasticNet()
pipe1=Pipeline([("Poly",poly),("el",el)])
pipe1.get_params()

{'memory': None,
 'steps': [('Poly', PolynomialFeatures()), ('el', ElasticNet())],
 'transform_input': None,
 'verbose': False,
 'Poly': PolynomialFeatures(),
 'el': ElasticNet(),
 'Poly__degree': 2,
 'Poly__include_bias': True,
 'Poly__interaction_only': False,
 'Poly__order': 'C',
 'el__alpha': 1.0,
 'el__copy_X': True,
 'el__fit_intercept': True,
 'el__l1_ratio': 0.5,
 'el__max_iter': 1000,
 'el__positive': False,
 'el__precompute': False,
 'el__random_state': None,
 'el__selection': 'cyclic',
 'el__tol': 0.0001,
 'el__warm_start': False}

In [78]:
params1={"Poly__degree":[2,3,4],"el__alpha":np.linspace(0.001,5,10),"el__l1_ratio":np.linspace(0.001,1,5)}
gcv1=GridSearchCV(pipe1,param_grid=params1,cv=kfold,verbose=3)
gcv1.fit(x,y)

Fitting 5 folds for each of 150 candidates, totalling 750 fits
[CV 1/5] END Poly__degree=2, el__alpha=0.001, el__l1_ratio=0.001;, score=0.913 total time=   0.0s
[CV 2/5] END Poly__degree=2, el__alpha=0.001, el__l1_ratio=0.001;, score=0.753 total time=   0.0s
[CV 3/5] END Poly__degree=2, el__alpha=0.001, el__l1_ratio=0.001;, score=0.820 total time=   0.0s
[CV 4/5] END Poly__degree=2, el__alpha=0.001, el__l1_ratio=0.001;, score=0.847 total time=   0.0s
[CV 5/5] END Poly__degree=2, el__alpha=0.001, el__l1_ratio=0.001;, score=0.811 total time=   0.0s
[CV 1/5] END Poly__degree=2, el__alpha=0.001, el__l1_ratio=0.25075;, score=0.913 total time=   0.0s
[CV 2/5] END Poly__degree=2, el__alpha=0.001, el__l1_ratio=0.25075;, score=0.754 total time=   0.0s
[CV 3/5] END Poly__degree=2, el__alpha=0.001, el__l1_ratio=0.25075;, score=0.820 total time=   0.0s
[CV 4/5] END Poly__degree=2, el__alpha=0.001, el__l1_ratio=0.25075;, score=0.847 total time=   0.0s
[CV 5/5] END Poly__degree=2, el__alpha=0.001, e

GridSearchCV(cv=KFold(n_splits=5, random_state=True, shuffle=True),
             estimator=Pipeline(steps=[('Poly', PolynomialFeatures()),
                                       ('el', ElasticNet())]),
             param_grid={'Poly__degree': [2, 3, 4],
                         'el__alpha': array([1.00000000e-03, 5.56444444e-01, 1.11188889e+00, 1.66733333e+00,
       2.22277778e+00, 2.77822222e+00, 3.33366667e+00, 3.88911111e+00,
       4.44455556e+00, 5.00000000e+00]),
                         'el__l1_ratio': array([0.001  , 0.25075, 0.5005 , 0.75025, 1.     ])},
             verbose=3)

In [79]:
gcv1.best_params_,gcv.best_score_

({'Poly__degree': 2,
  'el__alpha': np.float64(0.5564444444444444),
  'el__l1_ratio': np.float64(1.0)},
 np.float64(0.8354513478407453))

## for chemical dataset

In [81]:
import numpy as np
import pandas as pd
import os
from  sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import  SimpleImputer

In [84]:
data = pd.read_csv(r'../Datasets\ChemicalProcess.csv')
data.head()
data.isnull().sum()
data.columns
X, y = data.drop("Yield", axis=1), data["Yield"]
imputer = SimpleImputer()

In [85]:
el = ElasticNet()
pipe = Pipeline([('IMP', imputer), ('EL', el)])
pipe.get_params()

{'memory': None,
 'steps': [('IMP', SimpleImputer()), ('EL', ElasticNet())],
 'transform_input': None,
 'verbose': False,
 'IMP': SimpleImputer(),
 'EL': ElasticNet(),
 'IMP__add_indicator': False,
 'IMP__copy': True,
 'IMP__fill_value': None,
 'IMP__keep_empty_features': False,
 'IMP__missing_values': nan,
 'IMP__strategy': 'mean',
 'EL__alpha': 1.0,
 'EL__copy_X': True,
 'EL__fit_intercept': True,
 'EL__l1_ratio': 0.5,
 'EL__max_iter': 1000,
 'EL__positive': False,
 'EL__precompute': False,
 'EL__random_state': None,
 'EL__selection': 'cyclic',
 'EL__tol': 0.0001,
 'EL__warm_start': False}

In [86]:
kfold= KFold(n_splits=5, shuffle=True, random_state=25)
params= {'IMP__strategy':['mean', 'median'],'EL__alpha': np.linspace(0.001, 5, 10), 'EL__l1_ratio': np.linspace(0.001, 1, 5)}
gcv = GridSearchCV(pipe,param_grid= params, cv=kfold, verbose=0)
gcv.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=25, shuffle=True),
             estimator=Pipeline(steps=[('IMP', SimpleImputer()),
                                       ('EL', ElasticNet())]),
             param_grid={'EL__alpha': array([1.00000000e-03, 5.56444444e-01, 1.11188889e+00, 1.66733333e+00,
       2.22277778e+00, 2.77822222e+00, 3.33366667e+00, 3.88911111e+00,
       4.44455556e+00, 5.00000000e+00]),
                         'EL__l1_ratio': array([0.001  , 0.25075, 0.5005 , 0.75025, 1.     ]),
                         'IMP__strategy': ['mean', 'median']})

In [87]:
gcv.best_params_, gcv.best_score_
data_res = pd.DataFrame(gcv.cv_results_)
data_res.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_EL__alpha,param_EL__l1_ratio,param_IMP__strategy,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
14,0.015479,0.000887,0.002796,0.000257,0.556444,0.50050,mean,"{'EL__alpha': 0.5564444444444444, 'EL__l1_rati...",-1.813272,-1.863463,0.657922,-2.450661,0.601781,-0.973539,1.328330,1
15,0.017184,0.002526,0.002951,0.000311,0.556444,0.50050,median,"{'EL__alpha': 0.5564444444444444, 'EL__l1_rati...",-1.400029,-2.582499,0.667894,-2.296389,0.598793,-1.002446,1.391613,2
17,0.013870,0.003940,0.002796,0.000273,0.556444,0.75025,median,"{'EL__alpha': 0.5564444444444444, 'EL__l1_rati...",-2.232782,-1.336629,0.643482,-2.908921,0.608809,-1.045208,1.453006,3
23,0.020606,0.002598,0.003900,0.000689,1.111889,0.25075,median,"{'EL__alpha': 1.1118888888888887, 'EL__l1_rati...",-1.426206,-2.898656,0.661656,-2.375612,0.597653,-1.088233,1.480103,4
12,0.016744,0.001512,0.003029,0.000136,0.556444,0.25075,mean,"{'EL__alpha': 0.5564444444444444, 'EL__l1_rati...",-1.252978,-4.706876,0.706554,-0.789262,0.597010,-1.089110,1.963283,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,0.019944,0.000930,0.002674,0.000212,0.001000,0.50050,median,"{'EL__alpha': 0.001, 'EL__l1_ratio': 0.5005, '...",-19.479953,-4.793188,0.611581,-1.847157,0.409372,-5.019869,7.488677,96
7,0.019224,0.000692,0.002705,0.000286,0.001000,0.75025,median,"{'EL__alpha': 0.001, 'EL__l1_ratio': 0.75025, ...",-19.640313,-4.940581,0.607668,-1.869025,0.407595,-5.086931,7.547126,97
0,0.023283,0.005001,0.003566,0.000902,0.001000,0.00100,mean,"{'EL__alpha': 0.001, 'EL__l1_ratio': 0.001, 'I...",-19.127196,-6.201752,0.613165,-1.360482,0.410337,-5.133186,7.415641,98
9,0.021358,0.000811,0.003000,0.000185,0.001000,1.00000,median,"{'EL__alpha': 0.001, 'EL__l1_ratio': 1.0, 'IMP...",-20.044630,-5.061158,0.599994,-1.831783,0.403079,-5.186899,7.704035,99
